# This File Takes the Raw 2013 Loan Data and Cleans it for the app.py Regression

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from df_after_transform import df_after_transform
from sklearn import set_config
from sklearn.calibration import CalibrationDisplay
from sklearn.compose import (
    ColumnTransformer,
    make_column_selector,
    make_column_transformer,
)
from sklearn.decomposition import PCA
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_selection import (
    RFECV,
    SelectFromModel,
    SelectKBest,
    SequentialFeatureSelector,
    f_classif,
)
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso, LassoCV, LogisticRegression, Ridge
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    DetCurveDisplay,
    PrecisionRecallDisplay,
    RocCurveDisplay,
    classification_report,
    make_scorer,
)
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    cross_validate,
    train_test_split,
    cross_val_score,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    PolynomialFeatures,
    StandardScaler,
    MinMaxScaler,
    KBinsDiscretizer,
)
from sklearn.svm import LinearSVC
import streamlit as st


In [2]:
loans = pd.read_csv("inputs/2013_subsample.zip")

In [3]:
loans.shape

(134804, 33)

In [4]:
loans.info() # changing num_pipe_features from number to float64 to eliminate id as an option

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134804 entries, 0 to 134803
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    134804 non-null  int64  
 1   member_id             0 non-null       float64
 2   loan_status           134804 non-null  object 
 3   addr_state            134804 non-null  object 
 4   annual_inc            134804 non-null  float64
 5   application_type      134804 non-null  object 
 6   desc                  48728 non-null   object 
 7   dti                   134804 non-null  float64
 8   earliest_cr_line      134804 non-null  object 
 9   emp_length            128842 non-null  object 
 10  emp_title             126239 non-null  object 
 11  fico_range_high       134804 non-null  float64
 12  fico_range_low        134804 non-null  float64
 13  grade                 134804 non-null  object 
 14  home_ownership        134804 non-null  object 
 15  

## Omitted Columns ##
- **application_type**
    - all values are 'individual', so this variable doesn't provide any meaningful insights
- **desc**
    - There are 86,076 null fields and the existing data is not useful for this project
- **emp_title**
    - There are 83,424 unique values and it is a categorical variable
    - There are 8565 null fields
- **title**
    - There are 32,325 unique values and it is a categorical variable
    - Similar variable to purpose, which is consistently entered between 13 values
- **member_id**
    - all null values

In [5]:
loans = loans.drop(['application_type', 'desc', 'emp_title', 'title', 'member_id'], axis=1)

In [6]:
loans.shape

(134804, 28)

## Categorical Variables Converted to Numerical Variables ##
- **earliest_cr_line**
    - Entries are not numerical (ex. Sep-2003, Nov-1997)
    - Converting to numerical format (ex. 200309, 199711)
        - This gives models a quantitative way to interpret credit history
        - creates incremental issue between new years (ex. 200312 -> 200401)
        - instead: month number * 100/12
            - Jan - 0
            - Feb - 8.3333
            - Mar - 16.6667
            - Apr - 25
            - May - 33.3333
            - Jun - 41.6667
            - Jul - 50
            - Aug - 58.3333
            - Sep - 66.6667
            - Oct - 75
            - Nov - 83.3333
            - Dec - 91.6667
- **emp_length**
    - Entries are not numerical (ex. 4 years, 10+ years)
    - Converting to numerical format (ex. 4, 10)
        - This gives models a way to interpret employment length numerically rather than arbitrary categories
        - While 10 doesn't account for employment lengths beyond 10 years, it is the best way to quantify this data

In [7]:
loans[['earliest_cr_line', 'emp_length']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134804 entries, 0 to 134803
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   earliest_cr_line  134804 non-null  object
 1   emp_length        128842 non-null  object
dtypes: object(2)
memory usage: 2.1+ MB


In [8]:
loans[['earliest_cr_line', 'emp_length']]

,earliest_cr_line,emp_length
0,Sep-2003,3 years
1,Oct-1986,10+ years
2,Nov-1997,10+ years
3,Nov-1994,5 years
4,Dec-2009,4 years
...,...,...
134799,Aug-1997,10+ years
134800,Jun-1991,1 year
134801,May-1990,10+ years
134802,Aug-2000,9 years


In [9]:
loans['earliest_cr_line'] = pd.to_datetime(loans['earliest_cr_line'])
loans['earliest_cr_line'] = loans['earliest_cr_line'].dt.year + ((loans['earliest_cr_line'].dt.month * 100/12 - 100/12).round(4) / 100)

loans['emp_length'] = loans['emp_length'].astype(str)
loans['emp_length'] = loans['emp_length'].replace({'< 1 year': '0 years'}, regex=True)
loans['emp_length'] = loans['emp_length'].str.extract('(\d+)')[0].astype(float)



C:\Users\mason\AppData\Local\Temp\ipykernel_15088\3942494319.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  loans['earliest_cr_line'] = pd.to_datetime(loans['earliest_cr_line'])


In [10]:
loans[['earliest_cr_line', 'emp_length']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134804 entries, 0 to 134803
Data columns (total 2 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   earliest_cr_line  134804 non-null  float64
 1   emp_length        128842 non-null  float64
dtypes: float64(2)
memory usage: 2.1 MB


In [11]:
loans[['earliest_cr_line', 'emp_length']]

,earliest_cr_line,emp_length
0,2003.666667,3.0
1,1986.750000,10.0
2,1997.833333,10.0
3,1994.833333,5.0
4,2009.916667,4.0
...,...,...
134799,1997.583333,10.0
134800,1991.416667,1.0
134801,1990.333333,10.0
134802,2000.583333,9.0


In [12]:
loans.to_csv('inputs/final_2013_subsample.csv', index=False)

## Final Note ##

**emp_length (5962) and revol_util (78) have missing values, and should be included in the dashboard with warnings, as they only represent 4.4227% and 0.0579% of the observations respectively**


In [13]:
loans.isnull().sum()

id                         0
loan_status                0
addr_state                 0
annual_inc                 0
dti                        0
earliest_cr_line           0
emp_length              5962
fico_range_high            0
fico_range_low             0
grade                      0
home_ownership             0
initial_list_status        0
installment                0
int_rate                   0
issue_d                    0
loan_amnt                  0
mort_acc                   0
open_acc                   0
pub_rec                    0
pub_rec_bankruptcies       0
purpose                    0
revol_bal                  0
revol_util                78
sub_grade                  0
term                       0
total_acc                  0
verification_status        0
zip_code                   0
dtype: int64